# ACLO Project: Check-in Behavior Analysis

This notebook contains a complete data science pipeline analyzing gym/facility check-in behavior based on:
- Membership types and cardholder data (2020-2022)
- Special calendar days (academic calendar)
- Check-in timestamps (2017-2021)

**Goal:** Predict check-in patterns and identify key factors influencing member activity.

## 1. Setup and Data Loading

### Import Required Libraries

In [1]:
# Import all required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings

# For machine learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import KMeans
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, 
                             confusion_matrix, roc_auc_score, roc_curve,
                             r2_score, mean_squared_error, mean_absolute_error)

# For DAG visualization
import networkx as nx

# Settings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✓ All libraries imported successfully")

✓ All libraries imported successfully


### Load All Three Datasets

In [2]:
# Load the three datasets
membership_df = pd.read_csv('Data/Kaikki membership tyypit vuodet 2020-2022.csv')
special_days_df = pd.read_csv('Data/Yhdistetyt_p_iv_t__aikaj_rjestyksess__.csv')
checkins_df = pd.read_csv('Data/Check-in-times_sheet_2017-2021 kaikki checkin timet yhes.csv')

print("✓ All datasets loaded successfully")
print(f"\n1. Membership data shape: {membership_df.shape}")
print(f"2. Special days data shape: {special_days_df.shape}")
print(f"3. Check-ins data shape: {checkins_df.shape}")

✓ All datasets loaded successfully

1. Membership data shape: (32833, 4)
2. Special days data shape: (164, 4)
3. Check-ins data shape: (380785, 5)


### Dataset 1: Membership Data (2020-2022)

In [3]:
# Membership dataset - basic exploration
print("=" * 80)
print("MEMBERSHIP DATA OVERVIEW")
print("=" * 80)
print(f"\nShape: {membership_df.shape}")
print(f"\nFirst few rows:")
print(membership_df.head())
print(f"\nData types and non-null counts:")
print(membership_df.info())
print(f"\nMissing values per column:")
print(membership_df.isnull().sum())
print(f"\nBasic statistics:")
print(membership_df.describe(include='all'))

MEMBERSHIP DATA OVERVIEW

Shape: (32833, 4)

First few rows:
  Card Number Gender Male Gender Female          Description
0      206602         NaN        Female  ACLO Card 2020-2021
1      220965         NaN        Female  ACLO Card 2020-2021
2      221004        Male           NaN  ACLO Card 2020-2021
3      200011        Male           NaN  ACLO Card 2020-2021
4      187611        Male           NaN  ACLO Card 2020-2021

Data types and non-null counts:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32833 entries, 0 to 32832
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Card Number    32833 non-null  object
 1   Gender Male    14906 non-null  object
 2   Gender Female  17122 non-null  object
 3   Description    32833 non-null  object
dtypes: object(4)
memory usage: 1.0+ MB
None

Missing values per column:
Card Number          0
Gender Male      17927
Gender Female    15711
Description          0
dtype: in

### Dataset 2: Special Days (Academic Calendar)

In [4]:
# Special days dataset - basic exploration
print("=" * 80)
print("SPECIAL DAYS DATA OVERVIEW")
print("=" * 80)
print(f"\nShape: {special_days_df.shape}")
print(f"\nFirst few rows:")
print(special_days_df.head())
print(f"\nData types and non-null counts:")
print(special_days_df.info())
print(f"\nMissing values per column:")
print(special_days_df.isnull().sum())
print(f"\nBasic statistics:")
print(special_days_df.describe(include='all'))

SPECIAL DAYS DATA OVERVIEW

Shape: (164, 4)

First few rows:
         date                     event_name       event_type  \
0  2017-09-04  Getting Started (Orientation)  Getting started   
1  2017-09-05  Getting Started (Orientation)  Getting started   
2  2017-09-06  Getting Started (Orientation)  Getting started   
3  2017-09-26            EQUIS Accreditation    Special event   
4  2017-09-27            EQUIS Accreditation    Special event   

                   notes  
0  First-year onboarding  
1  First-year onboarding  
2  First-year onboarding  
3     Accreditation days  
4     Accreditation days  

Data types and non-null counts:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164 entries, 0 to 163
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        164 non-null    object
 1   event_name  164 non-null    object
 2   event_type  164 non-null    object
 3   notes       104 non-null    object
dtypes: o

### Dataset 3: Check-in Times (2017-2021)

In [5]:
# Check-ins dataset - basic exploration
print("=" * 80)
print("CHECK-INS DATA OVERVIEW")
print("=" * 80)
print(f"\nShape: {checkins_df.shape}")
print(f"\nFirst few rows:")
print(checkins_df.head())
print(f"\nData types and non-null counts:")
print(checkins_df.info())
print(f"\nMissing values per column:")
print(checkins_df.isnull().sum())
print(f"\nBasic statistics:")
print(checkins_df.describe(include='all'))

CHECK-INS DATA OVERVIEW

Shape: (380785, 5)

First few rows:
   Card nr          Affiliation  Gender      Subscription visit  \
0   168236  Medewerker RUG            M  Fitness Card 2016-2017   
1   132339            RUG             M  Fitness Card 2016-2017   
2   141170            RUG             M  Fitness Card 2016-2017   
3   141170            RUG             M  Fitness Card 2016-2017   
4   186817            RUG             M  Fitness Card 2016-2017   

    Check-in date  
0  8/1/2017 12:49  
1  8/1/2017 12:53  
2  8/1/2017 12:53  
3  8/1/2017 12:54  
4  8/1/2017 13:00  

Data types and non-null counts:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380785 entries, 0 to 380784
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   Card nr             380785 non-null  int64 
 1   Affiliation         355832 non-null  object
 2   Gender              380785 non-null  object
 3   Subscription visit  373

---

## Summary of Data Structure

### Key Findings:

**Dataset 1: Membership Data (32,833 rows, 4 columns)**
- Contains cardholder info with membership types from 2020-2022
- Key columns: `Card Number`, `Gender Male/Female`, `Description` (membership type)
- 21,871 unique cardholders
- Gender is split into two columns (needs cleaning)
- 5 membership types available

**Dataset 2: Special Days (164 rows, 4 columns)**
- Academic calendar events from 2017-2021
- Key columns: `date`, `event_name`, `event_type`, `notes`
- 6 event types: Exams (90 days), Getting started, Special event, etc.
- Most frequent: "Exam Week" (80 occurrences)

**Dataset 3: Check-ins (380,785 rows, 5 columns)**
- Individual check-in events from 2017-2021
- Key columns: `Card nr`, `Affiliation`, `Gender`, `Subscription visit`, `Check-in date`
- 241,617 unique timestamps
- Contains ~24,953 missing affiliations and ~7,774 missing subscription info
- Male members dominate (280,767 check-ins)

### Data Integration Plan:
1. **Join membership + check-ins** on Card Number/Card nr
2. **Enrich with special days** by matching check-in dates with event dates
3. **Extract time features** from check-in timestamps (hour, day of week, month, etc.)

### ⚠️ DECISION POINT: Target Variable

Before proceeding, we need to define the **target variable** for our predictive model.

**Possible options based on the data:**

1. **Classification: High vs Low Activity Member**
   - Target: Binary label (1 = active, 0 = inactive)
   - Example: "Did member check in ≥ X times in a given period?"
   
2. **Classification: Activity Level**
   - Target: Multi-class (e.g., Low / Medium / High activity)
   - Based on quartiles of check-in frequency

3. **Regression: Predict Check-in Count**
   - Target: Numeric value (total check-ins per member per period)
   
4. **Classification: Will Member Return Next Period?**
   - Target: Binary (1 = will check in next month/semester, 0 = won't)

**Questions to answer:**
- What threshold defines "active" vs "inactive"?
- What time period should we use for aggregation (monthly? semester? academic year)?
- Should we predict behavior or just classify existing patterns?

## ACLO case done with prompting
